# **Topic Modeling on Twitter Data**

Data: Twitter Timeline data of approximately 30 CCP MFAs and Ambassadors ~ 68K Tweets

Methodology:
-Use BERT-based sentence transformer to turn text into text embeddings where sentences can be compared to one another using text similarity
- Dimension reduction algorithm to enable 2D visualizations and use density-clustering to group similar tweets into themes.


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

In [ ]:
import torch
import io
from bertopic import BERTopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

## Data
For this example, we will use a pre-processed twitter dataset of approximately 62,000 Tweets in serialized form

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/taiwan_data_no-retweets-2023.pkl', 'rb') as f:
     data = pickle.load(f)

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [ ]:

from bertopic import BERTopic
model = BERTopic(embedding_model="all-MiniLM-L6-v2", calculate_probabilities=True, verbose=True, nr_topics="auto")
topics, probs = model.fit_transform(data)

import torch
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-model23')
torch.save(topics, '/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-topics23')
torch.save(probs, '/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-probs23')


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/159 [00:00<?, ?it/s]

2023-01-05 19:31:25,487 - BERTopic - Transformed documents to Embeddings
2023-01-05 19:31:50,757 - BERTopic - Reduced dimensionality
2023-01-05 19:31:53,127 - BERTopic - Clustered reduced embeddings


In [ ]:
#Load Trained Models
import torch
from bertopic import BERTopic
topics = torch.load("/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-topics23")
probs = torch.load("/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-probs23")
model = torch.load("/content/drive/MyDrive/Colab Notebooks/all_tweets_no-retweets-model23")

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. Our -1 Topic contains outliers of everything that was not fit into a topic cluster.

In [ ]:
freq = model.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,1654,-1_the_to_us_is
1,0,623,0_pla_eastern_command_theater
2,1,115,1_japan_japanese_question_bears
3,2,94,2_pelosi_consequences_measures_visit
4,3,93,3_dpp_progressive_democratic_authority
5,4,89,4_principle_one_iran_is
6,5,79,5_countermeasures_response_announces_nancy
7,6,74,6_ukraine_different_issue_question
8,7,73,7_diehard_die_secessionists_separatists
9,8,65,8_sovereignty_integrity_territorial_necessary


In [ ]:
model.get_topic(22)

[('government', 0.08105041926616867),
 ('sole', 0.07374956915065844),
 ('legal', 0.06937797658567754),
 ('representing', 0.06709316191871835),
 ('republic', 0.054418148368332434),
 ('whole', 0.050692380578656784),
 ('is', 0.03742892705421353),
 ('prc', 0.036583765282560236),
 ('there', 0.0365397581501132),
 ('inalienable', 0.036076547824552316)]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
model.visualize_topics()

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
model.visualize_hierarchy(top_n_topics=40)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
model.visualize_barchart(top_n_topics=30)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
model.visualize_heatmap(n_clusters=14, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [1]:
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(["amp"])

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 5))
model.update_topics(data, topics, vectorizer_model=vectorizer_model)

NameError: ignored

In [ ]:
for i in range(20):
  print(model.get_topic(i))   # We select topic that we viewed before

In [ ]:
model.visualize_barchart(top_n_topics=30)

## Dynamic Topic Modeling
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:


In [ ]:
import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/taiwan_dates_no_retweets.pkl', 'rb') as f:
with open('/content/drive/MyDrive/Colab Notebooks/taiwan_dates_no-retweets-2023.pkl', 'rb') as f:
     time = pickle.load(f)

In [ ]:
topics_over_time = model.topics_over_time(docs=data,
                                                #topics=topics,
                                                timestamps=time,
                                                global_tuning=True,
                                                evolution_tuning=True,
                                                nr_bins=15)

In [ ]:
topics_over_time.to_csv("Topics_over_Time.csv")

In [ ]:
model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
for i in range(20):
  print (model.get_topic(i))
  print ("-----------------------------------------------")

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
model.reduce_topics(docs = data, nr_topics=30)

In [ ]:
model.visualize_topics()

In [ ]:
topics_to_merge = [(9, 23),(10, 23)]
model.merge_topics(data, topics_to_merge)

In [ ]:
model.visualize_barchart(top_n_topics=30)

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = model.find_topics("war", top_n=5); similar_topics
for i in similar_topics:
  print (model.get_topic(i))
  print ('*'* 100)

[('ukraine', 0.03077364916884445), ('crisis', 0.013467485906718667), ('world', 0.012918522192140585), ('conflict', 0.012197217681990453), ('ukraine crisis', 0.011639992393534627), ('trying', 0.011396792702851292), ('peace', 0.009820661787067602), ('world largest government', 0.009321365888953532), ('world largest government sanctioned', 0.009321365888953532), ('sanctioned arms', 0.009321365888953532)]
****************************************************************************************************
[('continued', 0.03568512572170257), ('combat exercises', 0.03345685610201129), ('liberation army', 0.03214867407844738), ('people liberation army', 0.03214867407844738), ('people liberation', 0.03214867407844738), ('liberation', 0.03214867407844738), ('army continued', 0.031157839453573415), ('liberation army continued', 0.031157839453573415), ('people liberation army continued', 0.031157839453573415), ('army', 0.030771288900839803)]
*******************************************************

In [ ]:
similar_topics, similarity = model.find_topics("peace", top_n=5); similar_topics
for i in similar_topics:
  print (model.get_topic(i))
  print ('*'* 100)

[('ukraine', 0.03077364916884445), ('crisis', 0.013467485906718667), ('world', 0.012918522192140585), ('conflict', 0.012197217681990453), ('ukraine crisis', 0.011639992393534627), ('trying', 0.011396792702851292), ('peace', 0.009820661787067602), ('world largest government', 0.009321365888953532), ('world largest government sanctioned', 0.009321365888953532), ('sanctioned arms', 0.009321365888953532)]
****************************************************************************************************
[('cross', 0.02582309147510728), ('mainland', 0.022109306109225124), ('cross strait', 0.021045478360136715), ('development', 0.020385174038420543), ('strait relations', 0.0180898213347413), ('cross strait relations', 0.0180898213347413), ('relations', 0.017519808512502892), ('peaceful development cross strait relations', 0.016406116333336742), ('peaceful development cross strait', 0.016406116333336742), ('peaceful development cross', 0.016406116333336742)]
*********************************

In [ ]:
similar_topics, similarity = model.find_topics("friend", top_n=5); similar_topics
for i in similar_topics:
  print (model.get_topic(i))
  print ('*'* 100)

[('g7', 0.024489708908670388), ('summoned', 0.021664532973281646), ('eu', 0.02012180637468906), ('lodge', 0.019342394512652485), ('envoys', 0.018507654401681846), ('representations', 0.01805957624741557), ('japanese', 0.01760658057785293), ('statement', 0.017200678562215715), ('foreign', 0.01675666141671794), ('hideo', 0.015684238047352347)]
****************************************************************************************************
[('speakerpelosi', 0.02197296011765021), ('house speakerpelosi', 0.010031258872505855), ('visit', 0.009436043104747225), ('measures', 0.00873321538239754), ('china', 0.008709657200259282), ('insists', 0.007854355306488646), ('strong', 0.00720786674321088), ('house', 0.0068098702910350025), ('possible', 0.006808055364475668), ('resolutely', 0.00670648183759335)]
****************************************************************************************************
[('nancy pelosi visit', 0.0202352790368565), ('house', 0.01998802673746274), ('nancy pelos

In [ ]:
similar_topics, similarity = model.find_topics("enemy", top_n=5); similar_topics
for i in similar_topics:
  print (model.get_topic(i))
  print ('*'* 100)

[('north', 0.052922809967367736), ('south', 0.04182345405122017), ('bomber formations', 0.03764302650852982), ('formations', 0.03764302650852982), ('bomber', 0.03637925200614736), ('pla bomber formations', 0.034120096804766506), ('pla bomber', 0.034120096804766506), ('flew taiwan straits', 0.030509619847959555), ('flew taiwan straits north south', 0.030509619847959555), ('pla bomber formations flew taiwan', 0.030509619847959555)]
****************************************************************************************************
[('artillery', 0.027385333640139637), ('long range', 0.0246636017893247), ('range', 0.02287837365875299), ('drills', 0.022702076485681105), ('long range artillery', 0.022420723716465602), ('range artillery', 0.022420723716465602), ('missile', 0.021739766307248085), ('long', 0.021195908425306557), ('east', 0.02104916214107837), ('live', 0.020876836962127623)]
****************************************************************************************************
[('

In [ ]:
similar_topics, similarity = model.find_topics("covid", top_n=5); similar_topics
for i in similar_topics:
  print (model.get_topic(i))
  print ('*'* 100)

[('ccp', 0.04206675081284418), ('independent', 0.03503836069127927), ('taiwan', 0.03367429115810413), ('prc', 0.029536166356403794), ('taiwan independent', 0.0235119316701522), ('democratic', 0.02179976322362034), ('pa prc', 0.02030693064143259), ('pa', 0.020092468739289816), ('independent democratic', 0.019580785932159162), ('controlled', 0.01899636766728989)]
****************************************************************************************************
[('solomon', 0.04093651271003638), ('solomon islands', 0.04003545321237856), ('islands', 0.03998656346842121), ('china solomon islands', 0.035854308637840865), ('china solomon', 0.03578645644118919), ('security', 0.03295380363824823), ('cooperation', 0.03274049550190545), ('security cooperation', 0.025836899391061922), ('china', 0.022423048825779335), ('wang', 0.01943148497368293)]
****************************************************************************************************
[('solomon', 0.05144374297744865), ('islands', 0.

Cooperation,

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
model.save("reduced_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")